In [1]:

import threading
import sqlite3
from collections import deque
import time

# -------------------------
# 1️⃣ Database Setup (DDL & DML)
# -------------------------
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()
cursor.execute("CREATE TABLE Patients (id INT, name TEXT, diagnosis TEXT)")
cursor.executemany(
    "INSERT INTO Patients VALUES (?, ?, ?)",
    [(101, 'John', 'Diabetes'), (102, 'Jane', 'Hypertension')]
)
conn.commit()

# -------------------------
# 2️⃣ FIFO (Patient Queue)
# -------------------------
print("FIFO Patient Queue:")
queue = deque([101, 102])
while queue:
    print("Processing patient ID:", queue.popleft())

# -------------------------
# 3️⃣ LIFO (Lab Test Stack)
# -------------------------
print("\nLIFO Lab Stack:")
stack = ['Blood Test', 'X-Ray']
while stack:
    print("Processing:", stack.pop())

# -------------------------
# 4️⃣ Error Handling Function
# -------------------------
def get_patient(pid):
    try:
        cursor.execute("SELECT * FROM Patients WHERE id=?", (pid,))
        p = cursor.fetchone()
        if not p:
            raise ValueError("Patient not found!")
        print("Patient Info:", p)
    except Exception as e:
        print("Error:", e)

print("\nError Handling Example:")
get_patient(101)  # exists
get_patient(103)  # does not exist → triggers error

# -------------------------
# 5️⃣ Multithreading Monitoring
# -------------------------
def monitor(pid):
    for i in range(2):
        try:
            cursor.execute("SELECT name FROM Patients WHERE id=?", (pid,))
            p = cursor.fetchone()
            if not p:
                raise ValueError("Patient not found during monitoring")
            print(f"Monitoring {p[0]} - Heart rate {70+i} bpm")
            time.sleep(1)
        except Exception as e:
            print("Error:", e)

threads = []
for pid in [101, 103]:  # 103 does not exist → triggers error
    t = threading.Thread(target=monitor, args=(pid,))
    threads.append(t)
    t.start()

for t in threads:
    t.join()

conn.close()
print("\nAll tasks completed.")


FIFO Patient Queue:
Processing patient ID: 101
Processing patient ID: 102

LIFO Lab Stack:
Processing: X-Ray
Processing: Blood Test

Error Handling Example:
Patient Info: (101, 'John', 'Diabetes')
Error: Patient not found!
Error: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 4484 and this is thread id 4600.
Error: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 4484 and this is thread id 4600.
Error: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 4484 and this is thread id 1152.
Error: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 4484 and this is thread id 1152.

All tasks completed.
